In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes 

In [2]:
#export 
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
           ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x,m,s) : return (x -m)/s         

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
#NB: Use traning, not validation mean for validation set 
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(3.0614e-05), tensor(1.))

In [7]:
#export 
def test_near_zero(a, tol=1e-3): assert a.abs() < tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

# Foundation version 
## Basic Architecture 

In [10]:
# num hidden 
nh = 50

In [11]:
# simlified kaiming init/he init 
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1/math.sqrt(m))

In [13]:
# This should be ~ (0,1) (mean,std)
x_valid.mean(), x_valid.std()

(tensor(-0.0058), tensor(0.9924))

In [14]:
def lin(x,w,b): return x@w + b

In [15]:
t = lin(x_valid, w1, b1)

In [20]:
# Very important for the traning neural network 
t.mean(),t.std()

(tensor(-0.0681), tensor(1.0180))

In [17]:
## Important note, it is very import to have a output zero mean and 1 std as a output. So kaiming initalization is important for traning neural network 

In [21]:
# First layer is relu
def relu(x): return x.clamp_min(0.)

In [23]:

t = relu(lin(x_valid, w1, b1))

In [24]:
t.mean(), t.std()

(tensor(0.3689), tensor(0.5631))

# Read paper 2.2 
From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [25]:
# kaiming intialization / he init for relu 
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [26]:
w1.mean(), w1.std()

(tensor(0.0005), tensor(0.0505))

In [27]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.6091), tensor(0.8387))

In [28]:
#export 
from torch.nn import init

In [29]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [30]:
init.kaiming_normal_??

In [31]:
w1.mean(), w1.std()

(tensor(-0.0001), tensor(0.0505))

In [32]:
t.mean(), t.std()

(tensor(0.5205), tensor(0.8045))

In [33]:
w1.shape

torch.Size([784, 50])

In [34]:
import torch.nn

In [35]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [36]:
torch.nn.Linear.forward??

In [37]:
torch.nn.functional.linear??

In [38]:
torch.nn.Conv2d??

In [39]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [40]:
# what if 
def relu(x): return x.clamp_min(0.) - 0.5

In [41]:
#kaiming init 
w1 = torch.randn(m,nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.1371), tensor(0.8728))

In [42]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    
    return l3

In [43]:
%timeit -n 10 _=model(x_valid)

9.92 ms ± 3.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function MSE

In [45]:
model(x_valid).shape

torch.Size([10000, 1])

In [46]:
# Mse has to one unit vector so we will use squeez

In [54]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [48]:
y_train, y_valid = y_train.float(), y_valid.float()

In [49]:
preds = model(x_train)

In [50]:
preds.shape

torch.Size([50000, 1])

In [55]:
mse(preds, y_train)

tensor(21.4722)

### Gradient and backward pass 


In [56]:
def mse_grad(inp,targ):
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [57]:
def relu_grad(inp, out):
    #grad of relu with respect to input activation 
    inp.g = (inp>0).float() * out.g

In [58]:
def lin_grad(inp, out, w, b):
    # grad of relu with matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [59]:
def forward_and_backward(inp, targ):
    # forward pass : 
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2 
    # don't require
    loss = mse(out, targ)
    
    #backward pass
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)
    

In [60]:
forward_and_backward(x_train, y_train)

In [61]:
# save weights for testing later 
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

In [62]:
# Use pytorch autograd to track gradient
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [63]:
def forward(inp,targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [64]:
loss = forward(xt2, y_train)

In [65]:
loss.backward()

In [66]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)